In [1]:
import os
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split


In [2]:
users_df = pd.read_csv('Downloads/1481069814_relax_challenge/relax_challenge/takehome_users.csv', header=0, encoding = "ISO-8859-1")
users_engage_df = pd.read_csv('Downloads/1481069814_relax_challenge/relax_challenge/takehome_user_engagement.csv', header=0, encoding = "ISO-8859-1")

print(users_df.head())

    object_id        creation_time               name  \
0           1  2014-04-22 03:53:30     Clausen August   
1           2  2013-11-15 03:45:04      Poole Matthew   
2           3  2013-03-19 23:14:52  Bottrill Mitchell   
3           4  2013-05-21 08:09:28    Clausen Nicklas   
4           5  2013-01-17 10:14:20          Raw Grace   
5           6  2013-12-17 03:37:06      Cunha Eduardo   
6           7  2012-12-16 13:24:32       Sewell Tyler   
7           8  2013-07-31 05:34:02  Hamilton Danielle   
8           9  2013-11-05 04:04:24         Amsel Paul   
9          10  2013-01-16 22:08:03       Santos Carla   
10         11  2013-12-26 03:55:54     Paulsen Malthe   
11         12  2014-04-17 23:48:38    Mathiesen Lærke   
12         13  2014-03-30 16:19:38      Fry Alexander   
13         14  2012-10-11 16:14:33        Rivera Bret   
14         15  2013-07-16 21:33:54        Theiss Ralf   
15         16  2013-02-11 10:09:50         Engel René   
16         17  2014-04-09 14:39

In [3]:
# Convert string datetime to datetime format
users_engage_df.time_stamp = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in users_engage_df['time_stamp']]


In [4]:
import numpy as np
engaged_within_7 = []
for i in range(len(users_engage_df)):    
    time_stamp_start = users_engage_df.time_stamp[i]
    time_stamp_end = time_stamp_start + datetime.timedelta(days=7)
    user_id = users_engage_df.user_id[i]
    engaged_within_7.append(len(users_engage_df[(users_engage_df.user_id == user_id) & (users_engage_df.time_stamp >= 
                                                time_stamp_start) & (users_engage_df.time_stamp <= time_stamp_end)]))

In [5]:
users_engage_df['days_engaged_within_7'] = engaged_within_7
print(users_engage_df.head())

            time_stamp  user_id  visited  days_engaged_within_7
0  2014-04-22 03:53:30        1        1                      1
1  2013-11-15 03:45:04        2        1                      1
2  2013-11-29 03:45:04        2        1                      1
3  2013-12-09 03:45:04        2        1                      1
4  2013-12-25 03:45:04        2        1                      2
5  2013-12-31 03:45:04        2        1                      1
6  2014-01-08 03:45:04        2        1                      1
7  2014-02-03 03:45:04        2        1                      3
8  2014-02-08 03:45:04        2        1                      3
9  2014-02-09 03:45:04        2        1                      3
10 2014-02-13 03:45:04        2        1                      2
11 2014-02-16 03:45:04        2        1                      1
12 2014-03-09 03:45:04        2        1                      2
13 2014-03-13 03:45:04        2        1                      1
14 2014-03-31 03:45:04        2        1

In [6]:
engaged_users_id = users_engage_df[users_engage_df.days_engaged_within_7 > 2]['user_id'].unique()

In [7]:
#users_df['Engaged_User'] = [lambda x: True if x in engaded_users_id else 0 for x in users_df.object_id]
engaged_user = []
for x in users_df.object_id:
    if x in engaged_users_id:
        engaged_user.append(1)
    else:
        engaged_user.append(0)

In [8]:
users_df['Engaged_Users'] = engaged_user
print(users_df.describe())

         object_id  last_session_creation_time  opted_in_to_mailing_list  \
count  12000.00000                8.823000e+03              12000.000000   
mean    6000.50000                1.379279e+09                  0.249500   
std     3464.24595                1.953116e+07                  0.432742   
min        1.00000                1.338452e+09                  0.000000   
25%     3000.75000                1.363195e+09                  0.000000   
50%     6000.50000                1.382888e+09                  0.000000   
75%     9000.25000                1.398443e+09                  0.000000   
max    12000.00000                1.402067e+09                  1.000000   

       enabled_for_marketing_drip        org_id  invited_by_user_id  \
count                12000.000000  12000.000000         6417.000000   
mean                     0.149333    141.884583         5962.957145   
std                      0.356432    124.056723         3383.761968   
min                      0.0000

In [9]:
user_col_to_keep = ['object_id', 'creation_time','creation_source',
       'last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'invited_by_user_id', 'Engaged_Users']

users_df_ext = users_df[user_col_to_keep]
users_df_ext.fillna(0, inplace= True)
print(users_df_ext.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 8 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    12000 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
invited_by_user_id            12000 non-null float64
Engaged_Users                 12000 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 750.1+ KB
None


C:\Users\npshe\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [10]:

df = users_df_ext[['creation_time', 'last_session_creation_time']]

df['creation_time'] = pd.to_datetime(df['creation_time']).astype('int64')
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df) 
df.loc[:, :] = scaled_values

users_df_ext[['creation_time','last_session_creation_time']] = df
users_df_ext['invited_by_user_id'] = np.where(users_df_ext['invited_by_user_id'] < 1, 0, 1)
print(users_df_ext.head())

C:\Users\npshe\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\npshe\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\npshe\Anaconda3\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

   object_id  creation_time creation_source  last_session_creation_time  \
0          1       0.946796    GUEST_INVITE                    0.997198   
1          2       0.730340      ORG_INVITE                    0.995842   
2          3       0.401302      ORG_INVITE                    0.972660   
3          4       0.486746    GUEST_INVITE                    0.976566   
4          5       0.316995    GUEST_INVITE                    0.969176   

   opted_in_to_mailing_list  enabled_for_marketing_drip  invited_by_user_id  \
0                         1                           0                   1   
1                         0                           0                   1   
2                         0                           0                   1   
3                         0                           0                   1   
4                         0                           0                   1   

   Engaged_Users  
0              0  
1              1  
2              0 

C:\Users\npshe\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:

df_ind = users_df_ext.set_index('object_id')
df_dumb = pd.get_dummies(df_ind.select_dtypes(include=['object']), dummy_na=True)
user_dum_version = pd.concat([df_ind.select_dtypes(exclude=['object']), df_dumb], axis=1)

print(user_dum_version.head())

           creation_time  last_session_creation_time  \
object_id                                              
1               0.946796                    0.997198   
2               0.730340                    0.995842   
3               0.401302                    0.972660   
4               0.486746                    0.976566   
5               0.316995                    0.969176   
6               0.774170                    0.989556   
7               0.273338                    0.967151   
8               0.583863                    0.000000   
9               0.716660                    0.000000   
10              0.316304                    0.999833   
11              0.786517                    0.990050   
12              0.941083                    0.000000   
13              0.915998                    0.995813   
14              0.183085                    0.962906   
15              0.564227                    0.000000   
16              0.351238                    0.00

In [14]:
df_test = user_dum_version.loc[:, user_dum_version.columns != 'Engaged_Users']
y = user_dum_version.Engaged_Users
x_train,x_test,y_train,y_test = train_test_split(df_test,y,test_size=.2)

In [36]:


scores= []
#Train Classifier: Random Forest
clf = RandomForestClassifier(n_estimators=8)

#Fit classifier to Train data set
clf.fit(x_train,y_train)

#Predict Proba

feat_importance = clf.feature_importances_

data = {'column_names': df_test.columns, 'feat_importance': feat_importance}
feat_table = pd.DataFrame(data=data).sort_values(by='feat_importance',ascending=False)

print(feat_table)

                          column_names  feat_importance
1           last_session_creation_time         0.700287
0                        creation_time         0.284785
7    creation_source_PERSONAL_PROJECTS         0.003371
3           enabled_for_marketing_drip         0.003080
2             opted_in_to_mailing_list         0.002622
9   creation_source_SIGNUP_GOOGLE_AUTH         0.001517
5         creation_source_GUEST_INVITE         0.001266
6           creation_source_ORG_INVITE         0.001253
4                   invited_by_user_id         0.000999
8               creation_source_SIGNUP         0.000821
10                 creation_source_nan         0.000000


In [37]:
y_predict_test = clf.predict(x_test)


auc_test = metrics.roc_auc_score(y_test, y_predict_test)
print("AUC Score: ",auc_test)

AUC Score:  0.9174592789007886
